In [1]:
import logging
import sys

logger = logging.getLogger('BeamSearchRanker')
logger.setLevel(logging.DEBUG)
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# Prevent logs from propagating to the root logger
logger.propagate = False


In [2]:
from dataclasses import dataclass
import os
from dotenv import load_dotenv

load_dotenv()

@dataclass
class Config:
    # model configuration
    # base_model: str = "ollama_chat/qwen2.5:1.5b-instruct-q8_0"
    base_model: str = "ollama_chat/gemma2:2b-instruct-q8_0"
    # base_model: str = "ollama_chat/llama3.2:1b-instruct-q8_0"
    # base_model: str = "ollama_chat/exaone3.5:2.4b-instruct-q8_0"
    # base_model: str = "ollama_chat/granite3.1-dense:2b-instruct-q8_0"
    # base_model: str = "ollama_chat/granite3.1-moe:3b-instruct-q8_0"
    # base_model:str = "ollama_chat/llama3.2:3b-instruct-q8_0"
    temperature: float = 1.0
    # teacher_model: str = "openrouter/deepseek/deepseek-chat"
    # teacher_model: str = "openrouter/meta-llama/Llama-3.3-70B-Instruct-Turbo"
    teacher_model: str = "openrouter/qwen/qwen-2.5-72b-instruct"
    # teacher_model: str = "openrouter/qwen/qwq-32b-preview"
    teacher_temperature: float = 0.8

    reward_model: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"

    # dataset
    dataset: str = "HuggingFaceH4/MATH-500"

    # APIKEY (if using api for teacher)
    api_key: str | None = None


config = Config(
    api_key = os.environ["OPENROUTER_APIKEY"]
)

In [3]:
import dspy
from sentence_transformers.cross_encoder import CrossEncoder

# using cross encoder as reward model
reward_model = CrossEncoder(config.reward_model)

# small, locally hosted base model
lm = dspy.LM(config.base_model, api_base='http://localhost:11434', api_key='', temperature=config.temperature,  cache=False)
dspy.configure(lm=lm)

# teacher model for instruction proposal
teacher_lm = dspy.LM(config.teacher_model, api_key=config.api_key, temperature=config.temperature)

# Populate BM25S index

In [4]:
import bm25s
import Stemmer
from datasets import load_dataset, DatasetDict


dataset = load_dataset("BeIR/scidocs", "corpus")
corpus = dataset["corpus"]["text"]

# optional: create a stemmer
stemmer = Stemmer.Stemmer("english")

# Tokenize the corpus and only keep the ids (faster and saves memory)
corpus_tokens = bm25s.tokenize(corpus, stopwords="en", stemmer=stemmer)

# Create the BM25 model and index the corpus
retriever = bm25s.BM25()
retriever.index(corpus_tokens)

Split strings:   0%|          | 0/25657 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/25657 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/25657 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/25657 [00:00<?, ?it/s]

In [5]:
from functools import partial

def _query_bm25(corpus, query, k=5):
    query_tokens = bm25s.tokenize(query, stemmer=stemmer)
    results, scores = retriever.retrieve(query_tokens, k=k)
    return list(map(lambda i: corpus[i], results[0]))

query_bm25 = partial(_query_bm25, corpus)

query_bm25("hello world", k=1)

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

["1 Introduction Jackson (Jackson 95) recognises the vital difference between an application's domain and its code: two different worlds, each with its own language, experts, ways of thinking etc. A finished application forms the intersection between these worlds. The difficult job of the software engineer is to build a bridge between these worlds, at the same time as solving problems in both worlds."]

# DSPy Modules

In [6]:
GENERATE_TERMS = """
Help improve the search results from the BM25 index.
Analyze the previous search terms and results, and creatively write updated search terms to help find better results.
Reference the previous results to understand the kind of data in the index.
Create process steps that help resolve the data in the dataset and determine the next set of terms.

Focus on expanding the query terms so that the results do not overlap the previous results.
Generate new *unique* search terms to explore the index for different results that may apply to the query.
"""

In [7]:
import dspy

class GenerateNewSearchTerms(dspy.Signature):
    __doc__ = GENERATE_TERMS
    
    query: str = dspy.InputField(desc="The original search query")
    previous_search_terms: list[str] = dspy.InputField(desc="Previous list of search terms.")
    previous_results: list[str] = dspy.InputField(desc="The previous top search results.")
    #steps: list[str] = dspy.OutputField(desc="List of process steps to determine how to identify new terms")
    strategy: str = dspy.OutputField(desc="In one sentence, concisely mention how will you improve the search terms?")
    new_search_terms: list[str] = dspy.OutputField(desc="New search terms (tokens) to improve results.")

In [8]:
import hashlib
import heapq
from typing import List, Tuple, Dict
from collections import defaultdict
import logging
import sys

import dspy  # Ensure dspy is correctly imported based on your framework


def hash_result(result: str) -> str:
    """Generate a SHA-256 hash for a given result string."""
    return hashlib.sha256(result.encode('utf-8')).hexdigest()


class Scorer:
    """
    Handles scoring of results, caching scores, and maintaining a top-k heap.
    """
    def __init__(self, k: int):
        self.k = k
        self.score_cache: Dict[str, float] = defaultdict(float)
        self.heap: List[Tuple[float, str]] = []
        self.logger = logging.getLogger('Scorer')

    @classmethod
    def new(cls, k: int = 5):
        """Factory method to create a new Scorer instance."""
        return cls(k)

    def score_results(self, query: str, results: List[str], reward_model) -> List[float]:
        """
        Scores the given results using the reward model.

        Args:
            query (str): The search query.
            results (List[str]): List of result strings to score.
            reward_model: The model used to score the results.

        Returns:
            List[float]: Scores corresponding to the results.
        """
        results_to_score = [
            result for result in results
            if hash_result(result) not in self.score_cache
        ]

        # Initialize scores list with cached scores or None
        scores = []
        for result in results:
            result_hash = hash_result(result)
            if result_hash in self.score_cache:
                score = self.score_cache[result_hash]
                self.logger.debug(f"Cached Score: {score:.3f}, Hash: {result_hash}")
                self._push_to_heap(score, result, result_hash)
                scores.append(score)
            else:
                scores.append(None)  # Placeholder for new scores

        # Batch score the new results
        if results_to_score:
            try:
                query_result_pairs = [(query, result) for result in results_to_score]
                logger.debug(f"Scoring {len(results_to_score)} pairs...")
                new_scores = reward_model.predict(query_result_pairs)
            except Exception as e:
                self.logger.error(f"Error during batch scoring: {e}")
                new_scores = [0.0] * len(results_to_score)

            for result, score in zip(results_to_score, new_scores):
                result_hash = hash_result(result)
                self.score_cache[result_hash] = score
                self.logger.debug(f"New Score: {score:.3f}, Hash: {result_hash}")
                self._push_to_heap(score, result, result_hash)

            # Update the scores list with new scores
            score_iter = iter(new_scores)
            scores = [s if s is not None else next(score_iter) for s in scores]

        return scores

    def _push_to_heap(self, score: float, result: str, result_hash: str):
        """
        Push a result with its score to the heap, maintaining the top-k results.

        Args:
            score (float): The score of the result.
            result (str): The result string.
            result_hash (str): The hash of the result.
        """
        if len(self.heap) < self.k:
            heapq.heappush(self.heap, (score, result))
            self.logger.debug(f"Pushed to heap: Score: {score:.3f}, Hash: {result_hash}")
        else:
            if score > self.heap[0][0]:
                popped = heapq.heappushpop(self.heap, (score, result))
                popped_hash = hash_result(popped[1])
                self.logger.debug(
                    f"Popped from heap: Score: {popped[0]:.3f}, Hash: {popped_hash}"
                )
                self.logger.debug(
                    f"Pushed to heap: Score: {score:.3f}, Hash: {result_hash}"
                )

    def calculate_average_score(self, results: List[str]) -> float:
        """
        Calculate the average score of the given results.

        Args:
            results (List[str]): List of result strings.

        Returns:
            float: The average score.
        """
        cached_scores = [
            self.score_cache[hash_result(r)] for r in results
            if hash_result(r) in self.score_cache
        ]
        average_score = (
            sum(cached_scores) / len(cached_scores)
            if cached_scores else 0.0
        )
        self.logger.info(f"Calculated Average Score: {average_score:.3f}")
        return average_score

    def get_topk(self) -> Tuple[List[str], float]:
        """
        Retrieve the top-k results from the heap.

        Returns:
            Tuple[List[str], float]: Top-k results and their average score.
        """
        sorted_heap = sorted(self.heap, key=lambda x: x[0], reverse=True)
        top_results = [result for _, result in sorted_heap]
        average_top_score = (
            sum(score for score, _ in sorted_heap) / len(sorted_heap)
            if sorted_heap else 0.0
        )
        self.logger.info(f"Top-{self.k} Average Score: {average_top_score:.3f}")
        return top_results, average_top_score


In [9]:
class BeamSearchRanker(dspy.Module):
    def __init__(
        self,
        reward_model,
        depth: int = 3,
        expansions: int = 3,
        k: int = 5
    ):
        """
        Initialize the BeamSearchRanker.

        Args:
            reward_model: The model used to score the results.
            depth (int): Number of expansion steps.
            expansions (int): Number of expansions per depth step.
            k (int): Number of top results to keep.
        """
        super().__init__()
        self.depth = depth
        self.expansions = expansions
        self.k = k
        self.reward_model = reward_model

        # Initialize predictors and summarizer
        self.predictors = [
            dspy.ChainOfThought(GenerateNewSearchTerms) 
            for _ in range(depth)
        ]
        self.summarizer = dspy.ChainOfThought("query, context: list[str] -> answer")

        # Initialize logger
        self.logger = logging.getLogger('BeamSearchRanker')

    def forward(self, query: str) -> dspy.Prediction:
        """
        Perform the beam search ranking.

        Args:
            query (str): The initial search query.

        Returns:
            dspy.Prediction: The prediction containing terms, results, score, and summary.
        """
        self._initialize_search(query)

        # Instantiate a new Scorer for this forward pass
        scorer = Scorer.new(k=self.k)

        for depth_step in range(self.depth):
            self.logger.info(f"\n--- Depth Step {depth_step + 1}/{self.depth} ---")
            best_terms, best_results, top_score = self._process_depth_step(
                query, depth_step, self.expansions, self.predictors[depth_step], scorer
            )

            if best_terms:
                self.terms.update(" ".join(best_terms).split())
                self.results.update(best_results)
                self.logger.info(f"Total unique results so far: {len(self.results)}")
            else:
                self.logger.info("No better expansion found. Stopping early.")
                break

        final_results, final_score = scorer.get_topk()
        self.logger.info(f"Final Score: {final_score:.3f}")

        summary = self.summarizer(query=query, context=final_results)

        return dspy.Prediction(
            terms=list(self.terms),
            results=final_results,
            score=final_score,
            summary=summary
        )

    def _initialize_search(self, query: str):
        """
        Initialize the search by performing the initial search.
        """
        self.logger.info("\n--- Starting Forward Pass ---")

        # Initial search
        initial_results = query_bm25(query, k=self.k)
        self.terms = set(query.split())
        self.results = set(initial_results)
        self.logger.info(f"Initial Results: {len(initial_results)}")

    def _process_depth_step(
        self, 
        query: str, 
        depth_step: int, 
        expansions: int, 
        predictor, 
        scorer: Scorer
    ) -> Tuple[set, set, float]:
        """
        Process a single depth step with the given predictor and number of expansions.

        Args:
            query (str): The search query.
            depth_step (int): Current depth step.
            expansions (int): Number of expansions.
            predictor: The predictor to generate new search terms.
            scorer (Scorer): The scorer instance to handle scoring.

        Returns:
            Tuple[set, set, float]: Best search terms, best results, and top average score.
        """
        top_score = -float('inf')
        best_terms = set()
        best_results = set()

        for expansion in range(expansions):
            self.logger.info(
                f"--- Expansion {expansion + 1}/{expansions} ---"
            )
            new_search_terms, expanded_results = self._perform_expansion(query, predictor)
            self.logger.info(
                f"Expansion {expansion + 1}/{expansions} - Generated {len(expanded_results)} results"
            )

            # Handle results using Scorer
            scorer.score_results(query, expanded_results, self.reward_model)

            # Calculate average score for the current expansion
            average_expansion_score = scorer.calculate_average_score(expanded_results)
            self.logger.info(
                f"Expansion {expansion + 1} - Avg Score: {average_expansion_score:.3f}"
            )

            # Update best terms and results based on average score
            if average_expansion_score > top_score:
                self.logger.info(
                    f"Updating best terms and results (Expansion {expansion + 1})..."
                )
                top_score = average_expansion_score
                best_terms = set(new_search_terms)
                best_results = set(expanded_results)

        return best_terms, best_results, top_score

    def _perform_expansion(self, query: str, predictor):
        """
        Use the predictor to generate new search terms and perform an expanded search.

        Args:
            query (str): The current query.
            predictor: The predictor to generate new search terms.

        Returns:
            Tuple[List[str], List[str]]: New search terms and expanded results.
        """
        prediction = predictor(
            query=query,
            previous_search_terms=list(self.terms),
            previous_results=list(self.results)
        )
        self.logger.info(prediction.strategy)
        new_search_terms = prediction.new_search_terms
        new_query = " ".join(new_search_terms)
        expanded_results = query_bm25(new_query, k=self.k)
        return new_search_terms, expanded_results


In [10]:
b = BeamSearchRanker(reward_model=reward_model)

In [11]:
result = b("What are the consequences of being too fat?")

2024-12-23 21:04:16,904 - INFO - 
--- Starting Forward Pass ---


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

2024-12-23 21:04:17,007 - INFO - Initial Results: 5
2024-12-23 21:04:17,007 - INFO - 
--- Depth Step 1/3 ---
2024-12-23 21:04:17,007 - INFO - --- Expansion 1/3 ---
2024-12-23 21:04:19,398 - INFO - To improve search terms, I'll broaden the focus by exploring various effects and impacts of being overweight/obese on specific aspects of health, including physical health issues, emotional wellbeing, social dynamics, and psychological factors.  I'll avoid simply repeating previous terms but delve into different facets of the issue.


...

2024-12-23 21:04:48,142 - INFO - Expansion 2/3 - Generated 5 results
2024-12-23 21:04:48,143 - DEBUG - Scoring 4 pairs...
2024-12-23 21:04:48,148 - INFO - Expansion 2 - Avg Score: -11.033
2024-12-23 21:04:48,149 - INFO - --- Expansion 3/3 ---
2024-12-23 21:04:53,714 - INFO - A combination of general searches and domain-specific searches would likely yield useful results. Start with broad terms and then narrow down the focus based on specific keywords from each document.


Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

2024-12-23 21:04:53,724 - INFO - Expansion 3/3 - Generated 5 results
2024-12-23 21:04:53,725 - INFO - Expansion 3 - Avg Score: -8.821
2024-12-23 21:04:53,725 - INFO - Updating best terms and results (Expansion 3)...
2024-12-23 21:04:53,725 - INFO - Total unique results so far: 13
2024-12-23 21:04:53,725 - INFO - Final Score: -3.792


In [12]:
result

Prediction(
    terms=['are', 'for', 'fat', 'imaging', 'social', 'obesity', 'exercise', 'abdominal', 'individuals', 'appliances', 'almanac', 'genetic', 'metrics', 'being', 'people', 'lifestyles', 'tissue', 'insulin', 'determinants', 'habits', 'usage', 'conservation', 'overweight', 'efficiency', 'dataset', 'balance', 'anthropometric', 'lifestyle', 'factors', 'the', 'home', 'and', 'health', 'weight', 'effects', 'modifications', 'non-intrusive', 'lipase', 'energy', 'indicators', 'body', 'aging', 'stigma', 'in', 'status', 'consequences', 'ageing', 'fat?', 'characteristics', 'long-term', 'eating', 'What', '(NILM)', 'sensitivity', 'lipoprotein', 'load', 'disaggregation', 'visceral', 'techniques', 'household', 'distribution', 'monitoring', 'healthy', 'of', 'mental', 'adipose', 'minutely', 'power', 'too', 'appetite', 'assessment', 'image', 'composition', 'consumption'],
    results=['When prevention fails, medicinal treatment of obesity may become a necessity. Any strategic medicinal developme